# Import libraries

In [71]:
import pandas as pd
import numpy as np
import scipy.stats
import yfinance as yf
import datetime
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.plotting import plot_efficient_frontier
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# 1. Data Engineering - yfinance

In [72]:
# Variables
# Define ETF tickers and benchmark 
etfs = ['VWRA.L', 'VUAA.L', 'SPY4.L', 'PHYS', 'EXXY.MI', 'AGGU.L']
benchmark = '^GSPC'  # S&P 500

# Set the time frame for analysis
start_date = '2019-06-01'
end_date = '2023-12-31'


In [73]:
# Fetch historical price data for all ETFs and the benchmark
all_tickers = etfs + [benchmark]
data = yf.download(all_tickers, start=start_date, end=end_date)['Adj Close']

# Split the data into ETFs and benchmark
etf_prices = data[etfs]
benchmark_data = data[benchmark]

# Drop any rows with missing values
etf_prices.dropna(inplace=True)
benchmark_data.dropna(inplace=True)

# Display the first few rows of the data
print(etf_prices.head())
print(benchmark_data.head())

[*********************100%***********************]  7 of 7 completed

Ticker         VWRA.L     VUAA.L     SPY4.L   PHYS    EXXY.MI  AGGU.L
Date                                                                 
2019-07-23  80.000000  52.105610  56.520000  11.38  18.101999  5.4120
2019-07-24  80.263603  52.321918  57.150002  11.42  18.268000  5.4225
2019-07-25  79.925003  52.410404  57.395000  11.37  18.122000  5.4190
2019-07-26  80.065002  52.597218  57.560001  11.37  18.120001  5.4240
2019-07-29  80.035004  52.611965  57.404999  11.47  18.100000  5.4215
Date
2019-06-03    2744.449951
2019-06-04    2803.270020
2019-06-05    2826.149902
2019-06-06    2843.489990
2019-06-07    2873.340088
Name: ^GSPC, dtype: float64



C:\Users\User\AppData\Local\Temp\ipykernel_4612\2344532748.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# 2. Metrics


## 2.1 - Daily Returns and Volatility

Trying out arithmetic mean with log returns, attempting to assert the calculations made by PyPortfolioOpt library

In [74]:
# Calculate daily returns
log_returns = np.log(etf_prices / etf_prices.shift(1)).dropna()
mean_annual = log_returns.mean() * 252 # Annualise trading days
print("Numpy manual calculation of arithmetic mean")
print(mean_annual)
print()

# Asserting how PyPortfolio Opt calculates mean historical mean
print("PyPortfolioOpt library calculation of arithmetic mean")
print(expected_returns.mean_historical_return(etf_prices, compounding=False, log_returns=True))


Numpy manual calculation of arithmetic mean
Ticker
VWRA.L     0.091133
VUAA.L     0.126476
SPY4.L     0.095946
PHYS       0.078119
EXXY.MI    0.064243
AGGU.L    -0.002112
dtype: float64

PyPortfolioOpt library calculation of arithmetic mean
Ticker
VWRA.L     0.091133
VUAA.L     0.126476
SPY4.L     0.095946
PHYS       0.078119
EXXY.MI    0.064243
AGGU.L    -0.002112
dtype: float64


## 2.2 - VaR (Historical Estimation)

In [75]:
# Calculate VaR at 5% level
var_5 = log_returns.quantile(0.05)
print("VaR(5) for each ETF and benchmark:\n", var_5)

VaR(5) for each ETF and benchmark:
 Ticker
VWRA.L    -0.018923
VUAA.L    -0.019464
SPY4.L    -0.023551
PHYS      -0.016628
EXXY.MI   -0.016834
AGGU.L    -0.004970
Name: 0.05, dtype: float64


## 2.3 - CVaR (Historical Estimation)

Conditional VaR (also known as Expected Shortfall) is the average of the returns below the VaR threshold.

In [76]:
# Calculate CVaR
cvar_5 = log_returns[log_returns < var_5].mean()
print("CVaR(5) for each ETF and benchmark:\n", cvar_5)

CVaR(5) for each ETF and benchmark:
 Ticker
VWRA.L    -0.029464
VUAA.L    -0.030773
SPY4.L    -0.036941
PHYS      -0.023887
EXXY.MI   -0.026894
AGGU.L    -0.007226
dtype: float64


## 2.4 - Maximum Drawdown

Maximum drawdown measures the largest peak-to-trough decline in cumulative returns.

In [77]:
# Calculate cumulative returns
cumulative_returns = (1 + log_returns).cumprod()

# Calculate drawdowns
drawdowns = cumulative_returns / cumulative_returns.cummax() - 1

# Calculate maximum drawdown
max_drawdown = drawdowns.min()
print("Maximum drawdown for each ETF and benchmark:\n", max_drawdown)

Maximum drawdown for each ETF and benchmark:
 Ticker
VWRA.L    -0.345285
VUAA.L    -0.349274
SPY4.L    -0.442882
PHYS      -0.256297
EXXY.MI   -0.276953
AGGU.L    -0.157353
dtype: float64


## 2.5 - Covariance Matrix

In [78]:
S = risk_models.sample_cov(etf_prices, log_returns=True)

# Plot covariance matrix as a heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=S.values,  # Covariance values
        x=S.columns,  # ETF names (columns)
        y=S.columns,  # ETF names (rows)
        colorscale="reds",  # Choose a color scale
        colorbar=dict(title="Covariance"),
    )
)

# Customize the layout
fig.update_layout(
    title="Covariance Matrix Heatmap",
    xaxis_title="Assets",
    yaxis_title="Assets",
    width=600,
    height=600
)

fig.show()


# 3. Portfolio Optimisation 

Make use of PyPortfolioOpt to do optimisation with MPT and Efficient Frontier, compare with manual calucations

In [79]:
ef = EfficientFrontier(mean_annual, S)
weights = ef.max_sharpe()  # Maximize the Sharpe ratio
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

performance = ef.portfolio_performance(verbose=True)

OrderedDict({'VWRA.L': 0.0, 'VUAA.L': 0.43915, 'SPY4.L': 0.0, 'PHYS': 0.40582, 'EXXY.MI': 0.15502, 'AGGU.L': 0.0})
Expected annual return: 9.7%
Annual volatility: 12.5%
Sharpe Ratio: 0.78


# 4. Efficient Frontier
Plotting Efficient Frontier for given set of ETFs


In [80]:
# Calculate expected returns and the covariance matrix
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# Create the Efficient Frontier object
ef = EfficientFrontier(mu, S)

# Generate portfolios for the efficient frontier
ef_data = []
target_risks = []
target_returns = []
portfolio_weights = []

# Loop through different target returns to build the efficient frontier
for target_return in np.linspace(mu.min(), mu.max(), 50):
    try:
        ef.efficient_return(target_return)
        weights = ef.clean_weights()
        portfolio_weights.append(weights)
        target_risks.append(ef.portfolio_performance()[1])  # Standard deviation (volatility)
        target_returns.append(target_return)
    except Exception:
        pass

# Convert weights to a DataFrame for hover labels
weights_df = pd.DataFrame(portfolio_weights)

# Create the Plotly chart
fig = go.Figure()

# Add Efficient Frontier
fig.add_trace(
    go.Scatter(
        x=target_risks,
        y=target_returns,
        mode="lines+markers",
        name="Efficient Frontier",
        customdata=weights_df.values,  # Pass weights as custom data
        hovertemplate=(
            "Volatility: %{x:.2f}<br>"
            "Return: %{y:.2f}<br>"
            + "<br>".join([f"{etf}: %{{customdata[{i}]}}" for i, etf in enumerate(etfs)])
        ),
    )
)

# Add chart layout
fig.update_layout(
    title="Efficient Frontier with Portfolio Weights",
    xaxis_title="Risk (Volatility)",
    yaxis_title="Expected Return",
    width=1000,
    height=600,
)

# Show the chart
fig.show()

c:\Users\User\Desktop\Finance Resources\Personal Portfolio Optimisation\Personal-Portfolio-Optimisation\.venv\Lib\site-packages\pypfopt\expected_returns.py:32: UserWarning:

Some returns are NaN. Please check your price data.

